In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import torch.nn.init as init

### parameter ###
batchSize = 256

### parameter ###

### dataset ###

#Set normalizer
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
#Set transform function
transform_train = transforms.Compose(
    [transforms.RandomCrop(32),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     normalize])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

#set dataset  
trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_test)
#set loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchSize,
                                          shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize,
                                         shuffle=False, num_workers=2,pin_memory=True)
#set class label on dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#import matplotlib.pyplot as plt
#import numpy as np


Files already downloaded and verified
Files already downloaded and verified


In [2]:
#define NN

# def _weights_init(m):
#     classname = m.__class__.__name__
#     #print(classname)
#     if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
#         init.kaiming_normal_(m.weight)

class basicBlock(nn.Module):
    def __init__(self, inChannel, growthRate, layerDepth):
        super(basicBlock, self).__init__()
        self.layerDepth = layerDepth
        layers = []
        for i in range(layerDepth):
            layers.append(nn.BatchNorm2d(inChannel))
            layers.append(nn.Conv2d(inChannel, growthRate, kernel_size=3, padding = 1, bias=False))
            inChannel += growthRate
        self.moduleList = nn.ModuleList(layers)
        
    def forward(self, x):
        for i in range(self.layerDepth):
            out = F.relu(self.moduleList[2*i](x))
            out = self.moduleList[2*i + 1](out)    
            #concat
            x = torch.cat((x,out),dim=1)
        return x
        
        

class denseNet(nn.Module):
    def _theta(self,layerDepth):
        return int(layerDepth*self.theta)
    
    def __init__(self):
        super(denseNet, self).__init__()
        #parameter
        self.inChannel = 16
        self.growthRate = 12
        self.layerDepth = 12
        self.theta = 0.5 
        
        self.conv0 = nn.Conv2d(3, 16, kernel_size=3, padding = 1, bias = False)
        
        #transicion layer
        TLdepth1 = self.inChannel + (self.growthRate)*self.layerDepth
        self.TLbn1 = (nn.BatchNorm2d(TLdepth1))
        self.convTL1  = nn.Conv2d(TLdepth1, self._theta(TLdepth1), kernel_size = 1, bias = False)
        self.avgPool1 = nn.AvgPool2d(2, stride = 2)
        
        TLdepth2 = self._theta(TLdepth1) + (self.growthRate)*self.layerDepth
        self.TLbn2 = (nn.BatchNorm2d(TLdepth2))
        self.convTL2  = nn.Conv2d(TLdepth2, self._theta(TLdepth2), kernel_size = 1, bias = False)
        self.avgPool2 = nn.AvgPool2d(2, stride = 2)
        
        #dense blk
        #when cifar, layer's depth are same
        self.dense1 = basicBlock(self.inChannel, self.growthRate, self.layerDepth)
        self.dense2 = basicBlock(self._theta(TLdepth1), self.growthRate, self.layerDepth)
        self.dense3 =  basicBlock(self._theta(TLdepth2), self.growthRate, self.layerDepth)
        
        #classification Layer
        depthClass = self._theta(TLdepth2) +  (self.growthRate)*self.layerDepth
        self.GlobalAvgPool = nn.AvgPool2d(8, stride = 1)
        self.linear = nn.Linear(depthClass, 10)
        
        
#         self.apply(_weights_init)

    
    def forward(self, x):
        
        x = self.conv0(x)
        
        x = self.dense1(x)
        x = self.avgPool1(self.convTL1(F.relu(self.TLbn1(x))))
        x = self.dense2(x)
        x = self.avgPool2(self.convTL2(F.relu(self.TLbn2(x))))
        x = self.dense3(x)
        
        x = self.GlobalAvgPool(x)
        x = x.view(x.size(0), -1)
        #print(x)
        x = self.linear(x)
        return x

In [3]:
net = denseNet()
device = torch.device("cuda:0")
net.to(device)
summary(net,(3,32,32))
#use cuda instead cpu 

#loss function 
criterion = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
#eopch 100 -> lr = 0.01, epoch 150 -> lr = 0.001
decay_epoch = [100, 150]
step_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 12, 32, 32]           1,728
       BatchNorm2d-4           [-1, 28, 32, 32]              56
            Conv2d-5           [-1, 12, 32, 32]           3,024
       BatchNorm2d-6           [-1, 40, 32, 32]              80
            Conv2d-7           [-1, 12, 32, 32]           4,320
       BatchNorm2d-8           [-1, 52, 32, 32]             104
            Conv2d-9           [-1, 12, 32, 32]           5,616
      BatchNorm2d-10           [-1, 64, 32, 32]             128
           Conv2d-11           [-1, 12, 32, 32]           6,912
      BatchNorm2d-12           [-1, 76, 32, 32]             152
           Conv2d-13           [-1, 12, 32, 32]           8,208
      BatchNorm2d-14           [-1, 88,

In [4]:
#

print(net)


denseNet(
  (conv0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (TLbn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convTL1): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (avgPool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (TLbn2): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convTL2): Conv2d(224, 112, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (avgPool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (dense1): basicBlock(
    (moduleList): ModuleList(
      (0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(16, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Conv2d(28, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(40, eps=

In [5]:

#print all layer of NN




In [6]:

print('hing')
for epoch in range(200):
    net.train()
    running_loss = 0.0
    #print(epoch)
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data[0].to(device), data[1].to(device)
        #print(i)
        # 변화도(Gradient) 매개변수를 0으로 만들고
        

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
                  
    step_lr_scheduler.step()
    print('epoch : %5d loss : %.7f' %(epoch + 1, (running_loss /int(50000/batchSize))))
    
    correct = 0
    total = 0
    #net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))


hing
epoch :     1 loss : 1.6554533
Accuracy of the network on the 10000 test images: 47 %
epoch :     2 loss : 1.2637592
Accuracy of the network on the 10000 test images: 57 %
epoch :     3 loss : 1.0133643
Accuracy of the network on the 10000 test images: 66 %
epoch :     4 loss : 0.8432992
Accuracy of the network on the 10000 test images: 71 %
epoch :     5 loss : 0.7400968
Accuracy of the network on the 10000 test images: 74 %
epoch :     6 loss : 0.6416596
Accuracy of the network on the 10000 test images: 76 %
epoch :     7 loss : 0.5608484
Accuracy of the network on the 10000 test images: 79 %
epoch :     8 loss : 0.5057934
Accuracy of the network on the 10000 test images: 80 %
epoch :     9 loss : 0.4692093
Accuracy of the network on the 10000 test images: 82 %
epoch :    10 loss : 0.4321649
Accuracy of the network on the 10000 test images: 83 %
epoch :    11 loss : 0.4034762
Accuracy of the network on the 10000 test images: 83 %
epoch :    12 loss : 0.3709425
Accuracy of the ne

Accuracy of the network on the 10000 test images: 88 %
epoch :    97 loss : 0.0684203
Accuracy of the network on the 10000 test images: 88 %
epoch :    98 loss : 0.0683582
Accuracy of the network on the 10000 test images: 88 %
epoch :    99 loss : 0.0728281
Accuracy of the network on the 10000 test images: 88 %
epoch :   100 loss : 0.0705363
Accuracy of the network on the 10000 test images: 88 %
epoch :   101 loss : 0.0274835
Accuracy of the network on the 10000 test images: 90 %
epoch :   102 loss : 0.0105683
Accuracy of the network on the 10000 test images: 90 %
epoch :   103 loss : 0.0073821
Accuracy of the network on the 10000 test images: 90 %
epoch :   104 loss : 0.0057354
Accuracy of the network on the 10000 test images: 90 %
epoch :   105 loss : 0.0045929
Accuracy of the network on the 10000 test images: 90 %
epoch :   106 loss : 0.0041599
Accuracy of the network on the 10000 test images: 90 %
epoch :   107 loss : 0.0034478
Accuracy of the network on the 10000 test images: 90 %

epoch :   192 loss : 0.0006286
Accuracy of the network on the 10000 test images: 91 %
epoch :   193 loss : 0.0007126
Accuracy of the network on the 10000 test images: 90 %
epoch :   194 loss : 0.0006919
Accuracy of the network on the 10000 test images: 91 %
epoch :   195 loss : 0.0007020
Accuracy of the network on the 10000 test images: 91 %
epoch :   196 loss : 0.0006851
Accuracy of the network on the 10000 test images: 91 %
epoch :   197 loss : 0.0006237
Accuracy of the network on the 10000 test images: 91 %
epoch :   198 loss : 0.0006277
Accuracy of the network on the 10000 test images: 91 %
epoch :   199 loss : 0.0006251
Accuracy of the network on the 10000 test images: 91 %
epoch :   200 loss : 0.0006535
Accuracy of the network on the 10000 test images: 91 %


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 91 %


In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels =  data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 87 %
Accuracy of   car : 92 %
Accuracy of  bird : 100 %
Accuracy of   cat : 86 %
Accuracy of  deer : 76 %
Accuracy of   dog : 86 %
Accuracy of  frog : 83 %
Accuracy of horse : 91 %
Accuracy of  ship : 95 %
Accuracy of truck : 94 %


In [9]:
PATH = '../model/denseNetCIFAR10Rev1.pth'
torch.save(net.state_dict(), PATH)